# 进行KPCA的小实验

In [2]:
# 导入相关包
from sklearn.datasets import fetch_olivetti_faces
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 读取数据

In [3]:
# 读取图片
faces, target = fetch_olivetti_faces(return_X_y=True, shuffle=True)

## Perform kernel PCA

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import PCA
from time import time

X_train, X_test, y_train, y_test = train_test_split(
    faces,target, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
t0 = time()
transformer = KernelPCA(n_components=100, kernel='sigmoid')
#transformer = PCA(n_components=100)
X_re_train = transformer.fit_transform(X_train)
X_re_test = transformer.transform(X_test)
print("done in %0.3fs" % (time() - t0))
X_re_train.shape,X_re_test.shape

done in 0.032s


((320, 100), (80, 100))

In [24]:
# 构造SVM 运用rbf核，并寻找最优参数
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import RandomizedSearchCV

print("Fitting the classifier to the training set")
t0 = time()
param_grid = {
    "C": loguniform(1e3, 1e5),
    "gamma": loguniform(1e-4, 1e-1),
}
clf = RandomizedSearchCV(
    SVC(kernel="rbf", class_weight="balanced"), param_grid, n_iter=10)
clf = clf.fit(X_re_train, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
done in 1.528s
Best estimator found by grid search:
SVC(C=90402.09464883797, class_weight='balanced', gamma=0.05075362570632528)


In [25]:
# 预测准确率
y_pred = clf.predict(X_re_test)
right = np.sum(y_pred==y_test)
print("The Classification Accuracy is %.2f%%." %(right/y_test.shape[0]*100))

The Classification Accuracy is 98.75%.
